In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import scipy.stats
from IPython.display import Video, display, HTML
import re

sns.set()


def gethighest(vals, prefix="logall_"):
        m = np.argmax([int(re.search(f'{prefix}(\d+)', val).group(1)) for val in vals])
        return vals[m]

def read_csvx(path, *args, **kwargs):
    try:
        return pd.read_csv(str(path) +'.gz', *args, **kwargs)
    except Exception as e:
        return pd.read_csv(str(path), *args, **kwargs)

        
def readparams(file):
    params = {} 
    with open(file) as f:
        for line in f:
            line = line[:-1]  # remove \n
            if not line or line.startswith('#') or line.startswith('import'):
                continue
            key, val = line.split('=')
            if val.lower() == 'true' or val.lower() == 'false':
                params[key] = val.lower() == 'true'
            else:
                try:
                    params[key] = float(val)
                except ValueError:
                    params[key] = val
    return params

In [ ]:
import tqdm
params = []
prefix = '/home/pecoffet/remoterobo'

paths = sorted([]
                  + glob.glob(f"{prefix}/logs/lionscross2/*/*/*/*/*/rep00/", recursive=True)
                  )
#paths = sorted(glob.glob(f"{prefix}/logs/lions-nvar-*-2019-06-*/**/rep00/", recursive=True))
for path in tqdm.tqdm_notebook(paths):
    #print(path)
    try:
        logall = read_csvx(gethighest(glob.glob(path + 'logall*')), delimiter="\t")
    except Exception as e:
        print(e)
        continue
    curparam = readparams(glob.glob(path + 'properties*')[0])
    logall.query(f'nbOnOpp == {curparam["nOpti"]}')["curCoopNoCoef"].mean()
    nmod = logall['nbOnOpp'].mode()[0]
    coopopti = np.nanmean(logall.query(f"nbOnOpp == {curparam['nOpti']}")["curCoopNoCoef"])
    if np.isnan(coopopti):  # WARNING, bold choice !
        coopopti = 0
    coopmod =  np.nanmean(logall.query(f'nbOnOpp == {nmod}')["curCoopNoCoef"].mean())
    params.append({'path':path, 'params':curparam, 'coopopti': coopopti, 'coopmod': coopmod})



In [ ]:
diffkeys = set()
missingkeys = set()
for i in tqdm.tqdm_notebook(range(len(params))):
    for key in params[i]['params'].keys():
        try:
            if any(params[j]['params'][key] != params[i]['params'][key] for j in range(0, len(params) - 1)):
                diffkeys.add(key)
        except KeyError:
            missingkeys.add(key)

In [ ]:
excluded = ['gSeed', 'gLogFilename', 'path']
processed = []
for elem in params:
    param = elem['params']
    ess = param['meanA'] / param['nOpti']
    so = param['meanA'] + param['b'] / float(param['nOpti'])
    outdict = {'coopopti': (elem['coopopti'] - ess) / (so - ess) , 'truecoopopti': elem['coopopti'],
               'coopmod': (elem['coopmod'] - ess) / (so - ess) , 'truecoopmod': elem['coopmod']}
    for key in diffkeys:
        if key in excluded: continue
        outdict[key] = elem['params'][key]
    processed.append(outdict)
    for key in missingkeys:
        if key in excluded: continue
        outdict[key] = elem['params'].get(key, False)
df = pd.DataFrame(processed)
normalized_df=(df-df.mean())/df.std()

In [ ]:
df

In [ ]:
nopti = 2
sns.regplot(x='gNbOfPhysicalObjects', y='truecoopopti', data=df.query(f'nOpti == {nopti} and nTolerance == 0.2'))
plt.plot()
plt.axhline(5/nopti, label='defect', c='r')
plt.axhline(5+5*(nopti-1)/nopti, label='SO', c='b')
plt.legend()

In [ ]:
sns.regplot(x='nOpti', y='truecoopopti', data=df.query(f' nTolerance == 0.2 and gNbOfPhysicalObjects > 50'))
sm.ols(formula='truecoopopti ~ evaluationTime', data=df).fit().summary()

In [ ]:
import statsmodels.formula.api as sm

In [ ]:
#normalized_df['nbRobots'] = normalized_df['gInitialNumberOfRobots']
normalized_df['nbObj'] = normalized_df['gNbOfPhysicalObjects']

normalized_df

In [ ]:
result = sm.ols(formula="truecoopopti ~ gNbOfPhysicalObjects + nTolerance + nOpti + evaluationTime", data=df).fit()

In [ ]:
result.summary()

In [ ]:
df